In [ ]:
#code from https://github.com/chenyicai-0611/yolov5-PASCAL-VOC

!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt comet_ml  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CPU


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.7/107.7 GB disk)


In [ ]:
# Download COCO train
torch.hub.download_url_to_file('https://github.com/ultralytics/assets/releases/download/v0.0.0/coco2017labels.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

100%|██████████| 46.4M/46.4M [00:00<00:00, 101MB/s] 


In [ ]:
import os
import glob
#extract images with a certain label

def extract_detections(label_directory, label):
    """
    Get info about knives from coco

    label_directory: contains the label files
    label: label id for whatever object class

    Returns:
    list: List of filenames w object detections
    dict: Dictionary mapping filenames to their object detection coordinates
    """
    detected_files = []
    detections = {}

    # Get all txt files in the directory
    label_files = glob.glob(os.path.join(label_directory, '*.txt'))

    for label_file in label_files:
        with open(label_file, 'r') as f:
            print(label_file)
            lines = f.readlines()

            # Check if any line starts with the object label
            object_found = False
            current_detections = []

            for line in lines:
                values = line.strip().split()
                obj_label = int(float(values[0]))

                if obj_label == label:
                    object_found = True
                    x_center = float(values[1])
                    y_center = float(values[2])
                    width = float(values[3])
                    height = float(values[4])

                    current_detections.append({
                        'x_center': x_center,
                        'y_center': y_center,
                        'width': width,
                        'height': height
                    })

            if object_found:
                base_name = os.path.splitext(os.path.basename(label_file))[0]
                detected_files.append(base_name)
                detections[base_name] = current_detections

    return detected_files, detections


In [ ]:
knife_label = 43
knife_files, knife_coordinates = extract_detections("/content/datasets/coco/labels/train2017", knife_label)

In [ ]:
print(knife_files)

[]


In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs
!python train.py --img 640 --batch 16 --epochs 3 --data coco128.yaml --weights yolov5s.pt --cache

2024-12-09 19:17:56.862114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-09 19:17:56.883151: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-09 19:17:56.891369: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=, data=coco128.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, opti

In [ ]:
!zip -r ./runs/train/pretrained_coco_results.zip ./runs/train/exp2


	zip warning: name not matched: ./runs/train/exp2

zip error: Nothing to do! (try: zip -r ./runs/train/pretrained_coco_results.zip . -i ./runs/train/exp2)


In [ ]:
# Download COCO test
torch.hub.download_url_to_file('https://github.com/ultralytics/assets/releases/download/v0.0.0/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d ../datasets && rm tmp.zip  # unzip

100%|██████████| 780M/780M [00:06<00:00, 135MB/s]


In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half --verbose

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning /content/datasets/coco/val2017... 4952 images, 48 backgrounds, 0 corrupt: 100% 5000/5000 [00:03<00:00, 1404.59it/s]
val: New cache created: /content/datasets/coco/val2017.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   4% 7/157 [43:46<15:38:08, 375.25s/it]
Traceback (most recent call last):
  File "/content/yolov5/val.py", line 604, in <module>
    main(opt)
  File "/content/yolov5/val.py", line 575, in main
    run(**vars(opt

In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights yolov5s.pt --data coco.yaml --img 640 --half --verbose

val: data=/content/yolov5/data/coco.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
val: Scanning /content/datasets/coco/val2017.cache... 4952 images, 48 backgrounds, 0 corrupt: 100% 5000/5000 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95:   3% 5/157 [26:07<13:31:15, 320.23s/it]

In [ ]:
!pip install fiftyone

In [ ]:
#https://colab.research.google.com/github/voxel51/fiftyone-examples/blob/master/examples/quickstart.ipynb#scrollTo=4D6xZJxOa7ue
#https://cocodataset.org/#download

import fiftyone as fo
import fiftyone.zoo as foz

#dataset = fiftyone.zoo.load_zoo_dataset("coco-2017")
dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="test",
)

